In [1]:
import numpy as np
import pandas as pd
import luxpy as lp
from scipy.interpolate import LinearNDInterpolator
from itertools import product

np.set_printoptions(suppress=True)
np.set_printoptions(threshold=np.inf)

C:\Users\guym\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\guym\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\guym\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Import IES files

In [19]:
ies = lp.toolboxes.iolidfiles.read_lamp_data('ies/1.ies')

In [20]:
f_type = ies['photometric_type']
f_type

1

In [21]:
n_h_angles = len(ies['h_angs'])
n_v_angles = len(ies['v_angs'])

# create numpy array and insert intensity values(multiplied by candela_mult)
cd = np.zeros((n_h_angles, n_v_angles))

for i in range(n_h_angles):
    end_angles = n_v_angles*(i+1)
    cd[i] = ies['candela_values'][i*n_v_angles:end_angles]*ies['candela_mult']

In [22]:
ies['h_angs'], ies['v_angs']

(array([ 0.,  5., 15., 25., 35., 45., 46., 55., 65., 75., 85., 90.]),
 array([  0. ,   2.5,   5. ,   7.5,  10. ,  12.5,  15. ,  17.5,  20. ,
         22.5,  25. ,  27.5,  30. ,  32.5,  35. ,  37.5,  40. ,  42.5,
         45. ,  47.5,  50. ,  52.5,  55. ,  57.5,  60. ,  62.5,  65. ,
         66. ,  67.5,  70. ,  72.5,  75. ,  77.5,  80. ,  82.5,  85. ,
         87.5,  90. ,  92.5,  95. ,  97.5, 100. , 102.5, 105. , 107.5,
        110. , 112.5, 115. , 117.5, 120. , 122.5, 125. , 127.5, 130. ,
        132.5, 135. , 137.5, 140. , 142.5, 145. , 147.5, 150. , 152.5,
        155. , 157.5, 160. , 162.5, 165. , 167.5, 170. , 172.5, 175. ,
        177.5, 180. ]))

In [23]:
cd.shape

(12, 74)

### Broadcast and interpolate 

In [9]:
def broadcast(cd, h_angs):
    
    if max(h_angs)==360: # asymmetric, 360° is included
        full_h_angs = h_angs
        cd_0_360 = cd
        
    elif max(h_angs) > 180: # asymmetric, 360° not included
        full_h_angs = np.append(h_angs,360) # add 360°
        cd_0_360 = np.vstack((cd, cd[0])) # copy intensity data @0° to 360°
    
    elif max(h_angs) == 90: # quadrilateral symmetric
        h_angs_1 = 180 - h_angs[0:-1] # angle for 90° - 180° plane
        h_angs_2 = 180 + h_angs[1:] # angle for 0° - 180° plane
        h_angs_3 = 360 - h_angs[0:-1] # angle for 180° - 360° plane
        full_h_angs = np.sort(np.concatenate((h_angs, h_angs_1, h_angs_2, h_angs_3))) # full horizontal angles
        cd_90_180 = cd[:-1][::-1]  # intensity value for 90° - 180° plane
        cd_0_180 = np.vstack((cd, cd_90_180))  # intensity value for 0° - 180° plane
        cd_180_360 = cd_0_180[:-1][::-1]   # intensity value for 180° - 360° plane
        cd_0_360 = np.vstack((cd_0_180, cd_180_360))  # intensity value for 0° - 360° plane
        
    elif max(h_angs) == 180: # bilateral symmetric
        h_angs_1 = 360 - h_angs[:-1]
        full_h_angs = np.sort(np.concatenate((h_angs, h_angs_1))) # full horizontal angles
        cd_180_360 = cd[:-1][::-1] # intensity value for 180° - 360° plane
        cd_0_360 = np.vstack((cd, cd_180_360))  # intensity value for 0° - 360° plane
        
    elif max(h_angs) == 0: # axial symmetric
        full_h_angs = np.arange(0,360+1, 22.5) # standard indoor luminaire horizontal testing intervals
        cd_0_360 = np.repeat(cd, len(full_h_angs), axis=0) # intensity value for 0° - 360° plane 
        # changed from np.repeat([cd], len(full_h_angs), axis=0)
        
    return cd_0_360, full_h_angs

In [7]:
def interpolate(cd, h_angs, v_angs):
    
    if all(np.diff(h_angs)==np.diff(h_angs)[0]): # horizontal angles have equal intervals
        cd_inter = cd[:-1].transpose() # delete data @360°, transpose data for future use
        h_angs_inter = h_angs
        
    else: # horizontal angles have unequal intervals
        min_step = np.diff(h_angs).min() # find the minimum interval
        cd_T = cd.transpose() # transpose data for calculation
        
        if np.sum(np.diff(ies['h_angs'])%min_step) == 0:
            h_angs_inter = np.arange(0, 361, step=min_step) # create an array of angles with equal interval            
            cd_inter = np.zeros((len(v_angs), len(h_angs_inter))) 
            for i in range(len(v_angs)):
                cd_inter[i] = np.interp(h_angs_inter, h_angs, cd_T[i]) # linear interpolation
                
        else:
            h_angs_inter = np.arange(0, 361, step=1) # create an array of angles with equal interval
            cd_inter = np.zeros((len(v_angs), len(h_angs_inter))) 
            for i in range(len(v_angs)):
                # f = CubicSpline(h_angs, cd_T[i], bc_type='natural') # Cubic spline interpolation
                # cd_inter[i] = f(h_angs_inter)
                cd_inter[i] = np.interp(h_angs_inter, h_angs, cd_T[i]) # linear interpolation
            
        cd_inter = np.delete(cd_inter, -1, axis=1) # delete data @360°
        
    return cd_inter, h_angs_inter

In [24]:
cd_lumen, h_angs = broadcast(cd, ies['h_angs'])
cd_lumen, h_angs = interpolate(cd_lumen, h_angs, ies['v_angs'])

In [25]:
cd_lumen.shape

(74, 360)

In [16]:
h_angs.shape, ies['v_angs'].shape

((361,), (74,))

#### Add candela @360° horizontal back for candela interpolation

In [28]:
cd_full = np.vstack((cd_lumen.T, cd_lumen.T[0])) 

In [29]:
cd_full.shape

(361, 74)

### vector rotation in 3D

In [169]:
from scipy.spatial.transform import Rotation as R

def vector_rotate(tilt, orient, vector):
    
    # tilt angle -- 'y' axis, orientation angle-- 'z' axis

    r = R.from_euler('yz', [[-tilt, orient]], degrees=True)
    r.as_matrix()
    
    return r.apply(vector)

def cal_angle_vec(v1, v2):
    '''
    calculate the angle between two vectors
    '''
    
    # Add round to avoid invalid value encountered error
    A = np.round(np.dot(v1, v2)/(np.linalg.norm(v1) * np.linalg.norm(v2)),5) 
    ang = np.arccos(A)
    
    return ang

def project_vec_on_plane(v1, v_ortho):
    '''
    project vector to the luminaire horizontal plan, vector v_ortho is orthogonal vector to Plane Horizontal*
    
    '''
    proj_of_v1_on_v_ortho = v1 - (np.dot(v1, v_ortho)/np.linalg.norm(v_ortho)**2)*v_ortho
    
    return proj_of_v1_on_v_ortho

In [175]:
# linear data interpolation

x = np.radians(h_angs)
y = np.radians(ies['v_angs'])
z = cd_full.flatten()
interp = LinearNDInterpolator(list(product(x, y)), z)

In [171]:
def point_method_cal(origin:list[float], point:list[float], tilt, orientation):
    # vector1, luminaire center to nadir
    vector1 = [0,0,-1]

    # vector2, x-axis positive, horizontal angle 0°
    vector2 = [1,0,0]

    # vector 3, luminaire to calculation point
    vector3 = np.array(point) - np.array(origin)

    v1_r = vector_rotate(tilt, orientation, vector1)[0]
    v2_r = vector_rotate(tilt, orientation, vector2)[0]
    
    h_vec = project_vec_on_plane(vector3, v1_r)
    
    # v, h for point illuminance calculation, in radians
    p_v_ang = cal_angle_vec(v1_r, vector3)
    
    
    # use the vector cross product to determine if the horizontal angle is > 180°
    p_h_ang = cal_angle_vec(v2_r, h_vec)
    
    if np.cross(v2_r, h_vec)[2] < 0:
        p_h_ang = np.pi*2 - p_h_ang
   
    
       
    # theta for point method, in radians
    theta = cal_angle_vec(vector1, vector3)
    
    cd_p = interp(p_h_ang, p_v_ang)    
    fp = cd_p*np.cos(theta)/(np.linalg.norm(vector3))**2
    
    print(p_h_ang,p_v_ang, theta, cd_p, h_vec, v2_r)
    
    return np.round(fp,1)

In [173]:
a = np.arange(-5, 6,1)

point_illuminance = {}
for x, y in product(a,a):
    point_illuminance[(x,y)] = point_method_cal([0,0,6], [x,y,0], 10,45)

3.141592653589793 1.0416924000613672 0.8671529678610972 1934.7960730520429 [-5.57476584 -5.57476584 -1.39014595] [0.69636424 0.69636424 0.17364818]
3.0451187376815008 0.9914942699264412 0.817893296103056 1836.8501203519013 [-5.58984268 -4.58984268 -1.26922357] [0.69636424 0.69636424 0.17364818]
2.9309177420807258 0.941185810149312 0.7711064377877149 1660.0628515119245 [-5.60491953 -3.60491953 -1.14830119] [0.69636424 0.69636424 0.17364818]
2.79731647482791 0.8940111759002066 0.7314449971350676 1448.3189369412585 [-5.61999637 -2.61999637 -1.02737881] [0.69636424 0.69636424 0.17364818]
2.6450398113985054 0.8541495462693961 0.7044003628084701 1265.8666337225206 [-5.63507322 -1.63507322 -0.90645643] [0.69636424 0.69636424 0.17364818]
2.47820390398233 0.8258396002875474 0.694740274989304 1143.6832768974755 [-5.65015006 -0.65015006 -0.78553404] [0.69636424 0.69636424 0.17364818]
2.304776138837295 0.8119979061553548 0.7044003628084701 1086.9479994515432 [-5.66522691  0.33477309 -0.66461166] [

In [174]:
point_illuminance

{(-5, -5): 14.6,
 (-5, -4): 16.3,
 (-5, -3): 17.0,
 (-5, -2): 16.6,
 (-5, -1): 15.6,
 (-5, 0): 14.4,
 (-5, 1): 13.4,
 (-5, 2): 12.6,
 (-5, 3): 11.7,
 (-5, 4): 11.0,
 (-5, 5): 10.1,
 (-4, -5): 16.3,
 (-4, -4): 17.4,
 (-4, -3): 17.4,
 (-4, -2): 16.6,
 (-4, -1): 15.7,
 (-4, 0): 14.9,
 (-4, 1): 13.9,
 (-4, 2): 12.8,
 (-4, 3): 11.7,
 (-4, 4): 10.7,
 (-4, 5): 9.9,
 (-3, -5): 17.0,
 (-3, -4): 17.4,
 (-3, -3): 16.8,
 (-3, -2): 16.0,
 (-3, -1): 15.5,
 (-3, 0): 15.0,
 (-3, 1): 14.1,
 (-3, 2): 12.8,
 (-3, 3): 11.6,
 (-3, 4): 10.4,
 (-3, 5): 9.6,
 (-2, -5): 16.6,
 (-2, -4): 16.6,
 (-2, -3): 16.0,
 (-2, -2): 15.4,
 (-2, -1): 15.1,
 (-2, 0): 14.4,
 (-2, 1): 13.3,
 (-2, 2): 12.6,
 (-2, 3): 11.5,
 (-2, 4): 10.3,
 (-2, 5): 9.4,
 (-1, -5): 15.5,
 (-1, -4): 15.7,
 (-1, -3): 15.5,
 (-1, -2): 15.1,
 (-1, -1): 14.2,
 (-1, 0): 13.0,
 (-1, 1): 12.0,
 (-1, 2): 11.6,
 (-1, 3): 11.2,
 (-1, 4): 10.3,
 (-1, 5): 9.3,
 (0, -5): 14.4,
 (0, -4): 14.9,
 (0, -3): 15.0,
 (0, -2): 14.4,
 (0, -1): 13.0,
 (0, 0): 11.6,
 (0,

### Shaun's results

#### 1.ies, [0,0,6], orientation 45deg, tilt 10deg. inserted at photometric center. 10ft x 10ft calc grid

<img src="img/7.png" width="500">